<a href="https://colab.research.google.com/github/herculeslyndel/hugo-continuous-delivery-demo/blob/master/AWS_ECU_spot_pricing_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import core python libraries for retrieving and analysis of AWS data
import pandas as pd
import numpy as np
import boto3
import json
from sklearn.cluster import KMeans

# import visualization libraries, and do some configuration for jupyter notebook
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# 1 - get the complete list of all AWS services for all regions
# https://aws.amazon.com/blogs/aws/new-aws-price-list-api/

# import current master AWS index for all services
import requests
offers = requests.get('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/index.json')
print("master AWS index downloaded", "\n")

master AWS index downloaded 



In [12]:
offers.json()['offers']['AmazonEC2']['currentVersionUrl']

'/offers/v1.0/aws/AmazonEC2/current/index.json'

In [3]:
# 2- Use that AWS index to get the current offers list for EC2 as of today
# hard coded EC2 prices:  offers = requests.get('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.json')

# get the current URL `suffix` for EC2 pricing from the AWS master index
ec2_offer_path = offers.json()['offers']['AmazonEC2']['currentVersionUrl']
print("EC2 offer path deciphered", "\n")

# construct the complete URL for current EC2 pricing, and retrieve the json file
ec2pricingURL = 'https://pricing.us-east-1.amazonaws.com%s' % ec2_offer_path
print(ec2pricingURL, "\n")

ec2offers = requests.get('https://pricing.us-east-1.amazonaws.com%s' % ec2_offer_path).json()
print("current EC2 offers downloaded", "\n")

EC2 offer path deciphered 

https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.json 

current EC2 offers downloaded 



In [16]:
ec2offers.keys()

dict_keys(['formatVersion', 'disclaimer', 'offerCode', 'version', 'publicationDate', 'products', 'terms'])

In [18]:
# 3 - Dynamically, get the actual price list we want
# Pricing information can be accessed by URLs, each structured as follows:
# https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/{offer_code}/current/index.{format}
import requests

#we want the csv, so we have to trim the `index.json` filename off the pricing url
url_base=ec2pricingURL[:-10]

print(url_base)

pricing_csv_url = url_base + 'index.csv'

print(pricing_csv_url)

# write content to csv.  This will fail if you have a prior version of the file open in an editor or spreadsheet
r = requests.get(pricing_csv_url, stream=True)

https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/
https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.csv


In [24]:
with open('ec2-prices.csv', 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)

print("ec2_prices.csv saved to disk.","\n")

ec2_prices.csv saved to disk. 



In [ ]:
# construct set of unique EC2 Instance Types, and do a little cleanup
# count = 0
# uniq = set()
# for sku, data in ec2offers['products'].items():
#    try:
#        count += 1
#        # print(count, sku, data['productFamily'])
#        if data['productFamily'] == 'Compute Instance':
#            # Add EC2 Instance to the set
#            uniq.add(data['attributes']['instanceType'])
#          
#        elif data['productFamily'] == 'Compute':
#            # Add AWS misspelled instanceType for EC2 Instance to the set
#            uniq.add(data['attributes']['instanceType'])
#        
#        else:
#            # skip anything that's not an EC2 Instance  
#            continue
#    except KeyError:
#        # some AWS JSON entries don't have all their keys
#        pass
#        
#for instancetype in sorted(uniq):
#    print(instancetype)

In [63]:
# read the saved data into pandas
# Be sure to skip the first 5 rows of information above row 5 which has the column headers; and get rid of NaN which causes mixed type errors

pricing_df = pd.read_csv("ec2-prices.csv", skiprows = 5, low_memory=False)
pricing_df.set_index(['SKU', 'Location','Instance Type'])

print("number of rows in this dataframe: ", len(pricing_df), "\n")
pricing_df.head()

number of rows in this dataframe:  1603820 



,SKU,OfferTermCode,RateCode,TermType,PriceDescription,EffectiveDate,StartingRange,EndingRange,Unit,PricePerUnit,Currency,LeaseContractLength,PurchaseOption,OfferingClass,Product Family,serviceCode,Location,Location Type,Instance Type,Current Generation,Instance Family,vCPU,Physical Processor,Clock Speed,Memory,Storage,Network Performance,Processor Architecture,Storage Media,Volume Type,Max Volume Size,Max IOPS/volume,Max IOPS Burst Performance,Max throughput/volume,Provisioned,Tenancy,EBS Optimized,Operating System,License Model,Group,...,From Location,From Location Type,To Location,To Location Type,usageType,operation,CapacityStatus,Dedicated EBS Throughput,ECU,Elastic Graphics Type,Enhanced Networking Supported,GPU,GPU Memory,Instance,Instance Capacity - 10xlarge,Instance Capacity - 12xlarge,Instance Capacity - 16xlarge,Instance Capacity - 18xlarge,Instance Capacity - 24xlarge,Instance Capacity - 2xlarge,Instance Capacity - 32xlarge,Instance Capacity - 4xlarge,Instance Capacity - 8xlarge,Instance Capacity - 9xlarge,Instance Capacity - large,Instance Capacity - medium,Instance Capacity - metal,Instance Capacity - xlarge,instanceSKU,Intel AVX Available,Intel AVX2 Available,Intel Turbo Available,Normalization Size Factor,Physical Cores,Pre Installed S/W,Processor Features,Product Type,Resource Type,serviceName,Volume API Name
0,YQHNG5NBWUE3D67S,4NA7Y494T4,YQHNG5NBWUE3D67S.4NA7Y494T4.6YS6EN2CT7,Reserved,"Red Hat Enterprise Linux (Amazon VPC), m4.xlar...",2017-04-30,0.0,inf,Hrs,0.1963,USD,1yr,No Upfront,standard,Compute Instance,AmazonEC2,US East (Ohio),AWS Region,m4.xlarge,Yes,General purpose,4.0,Intel Xeon E5-2676 v3 (Haswell),2.4 GHz,16 GiB,EBS only,High,64-bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dedicated,NaN,RHEL,No License required,NaN,...,NaN,NaN,NaN,NaN,USE2-DedicatedUsage:m4.xlarge,RunInstances:0010,Used,750 Mbps,13,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,8.0,NaN,NaN,Intel AVX; Intel AVX2; Intel Turbo,NaN,NaN,Amazon Elastic Compute Cloud,NaN
1,FS9HBBMCNT422SDM,38NPMPTW36,FS9HBBMCNT422SDM.38NPMPTW36.6YS6EN2CT7,Reserved,Windows with SQL Server Enterprise (Amazon VPC...,2020-04-01,0.0,inf,Hrs,22.0310,USD,3yr,Partial Upfront,standard,Compute Instance (bare metal),AmazonEC2,AWS GovCloud (US-West),AWS Region,r5dn.metal,Yes,Memory optimized,96.0,Intel Xeon Platinum 8259 (Cascade Lake),3.1 GHz,768 GiB,4 x 900 NVMe SSD,100 Gigabit,64-bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dedicated,NaN,Windows,No License required,NaN,...,NaN,NaN,NaN,NaN,UGW1-DedicatedUsage:r5dn.metal,RunInstances:0102,Used,12000 Mbps,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,192.0,NaN,SQL Ent,NaN,NaN,NaN,Amazon Elastic Compute Cloud,NaN
2,AF4M9XS62QDH5P8S,BPH4J8HBKS,AF4M9XS62QDH5P8S.BPH4J8HBKS.6YS6EN2CT7,Reserved,"Linux/UNIX (Amazon VPC), i3.xlarge reserved in...",2017-04-30,0.0,inf,Hrs,0.1950,USD,3yr,No Upfront,standard,Compute Instance,AmazonEC2,Asia Pacific (Tokyo),AWS Region,i3.xlarge,Yes,Storage optimized,4.0,Intel Xeon E5-2686 v4 (Broadwell),2.3 GHz,30.5 GiB,1 x 950 NVMe SSD,Up to 10 Gigabit,64-bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dedicated,NaN,Linux,No License required,NaN,...,NaN,NaN,NaN,NaN,APN1-DedicatedUsage:i3.xlarge,RunInstances,Used,850 Mbps,16,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,8.0,NaN,NaN,Intel AVX; Intel AVX2; Intel Turbo,NaN,NaN,Amazon Elastic Compute Cloud,NaN
3,TDQJ4JGM4HQ9852M,R5XV2EPZQZ,TDQJ4JGM4HQ9852M.R5XV2EPZQZ.2TG2D8R56U,Reserved,Upfront Fee,2018-03-31,NaN,NaN,Quantity,6972.0000,USD,3yr,Partial Upfront,convertible,Compute Instance,AmazonEC2,US East (N. Virginia),AWS Region,m4.large,Yes,General purpose,2.0,Intel Xeon E5-2676 v3 (Haswell),2.4 GHz,8 GiB,EBS only,Moderate,64-bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dedicated,NaN,Linux,No License required,NaN,...,NaN,NaN,NaN,NaN,DedicatedUsage:m4.large,RunInstances:0004,Used,450 Mbps,6.5,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [64]:
print("number of rows before nan cleanup: ", len(pricing_df), "\n")

number of rows before nan cleanup:  1603820 



In [65]:
# Let's get rid of Null instance types, things that are not 'hourly' pricing, and things outside our region of interest
# AWS will reject request if this asks for too much, so to get other regions we would need to iterate over values
pricing_df = pricing_df[(pricing_df["Instance Type"].notnull()) & (pricing_df["Unit"] == "Hrs") & (pricing_df["Location"] == "US East (N. Virginia)")& (pricing_df["ECU"] != "Variable")]
print("number of rows after nan cleanup: ", len(pricing_df), "\n")

number of rows after nan cleanup:  65810 



In [61]:
pricing_df['Instance Type']

7           m5ad.2xlarge
26         r5ad.12xlarge
40          inf1.6xlarge
50             r3.xlarge
60            g2.2xlarge
               ...      
1603740     r5n.24xlarge
1603777     r5n.16xlarge
1603778        m3.medium
1603780      r6g.2xlarge
1603817     m5d.24xlarge
Name: Instance Type, Length: 65810, dtype: object

In [72]:
# memory currently stored as a string
# create new derivative column, to story memory amounts without units, also take out commmas
pricing_df["Memory_GiB"] = pricing_df["Memory"]
pricing_df["Memory_GiB"].replace(regex=True,inplace=True,to_replace=r' GiB',value=r'')
pricing_df["Memory_GiB"].replace(regex=True,inplace=True,to_replace=r',',value=r'')
pricing_df["Memory_GiB"].replace(regex=True,inplace=True,to_replace=r' ',value=r'')

# now cast this new column to float so we can use it as a number going forward
pricing_df['Memory_GiB'] = pricing_df['Memory_GiB'].astype(float)

# ECU currently stored as a string since some values in csv originally contained the value 'variable'
# now cast this new column to float so we can use it as a number going forward
pricing_df['ECU'] = pricing_df['ECU'].astype(float)

# clock speed currently stored as a string
# create new derivative column, to story clock speeds without units, also take out commmas
pricing_df["Clock Speed GHz"] = pricing_df["Clock Speed"]
pricing_df["Clock Speed GHz"].replace(regex=True,inplace=True,to_replace=r'Ghz',value=r'')
pricing_df["Clock Speed GHz"].replace(regex=True,inplace=True,to_replace=r' GHz',value=r'')
pricing_df["Clock Speed GHz"].replace(regex=True,inplace=True,to_replace=r'Up to ',value=r'')
pricing_df["Clock Speed GHz"].replace(regex=True,inplace=True,to_replace=r' ',value=r'')

# now cast this new column to float so we can use it as a number going forward
pricing_df['Clock Speed GHz'] = pricing_df['Clock Speed GHz'].astype(float)


# let's capture the different ways of looking at pricing vs. compute capacity, and compare them.
pricing_df["Price_vCPU"] = pricing_df["PricePerUnit"]/pricing_df["vCPU"]
pricing_df["Price_ECU"] = pricing_df["PricePerUnit"]/pricing_df["ECU"]
pricing_df["Price_vCPU_ECU_ratio"] = round(pricing_df["Price_vCPU"]/pricing_df["Price_ECU"],1)

In [73]:
# put all the names of Instance types into a dictionary, so we can pass that to boto3 and retrieve spot pricing
names = pricing_df["Instance Type"].to_dict()

In [74]:
# show a short sample of Instance Types, to check what we're working with
list(names.values())[0:5]

['m5ad.2xlarge', 'r5ad.12xlarge', 'inf1.6xlarge', 'r3.xlarge', 'g2.2xlarge']

In [75]:
# and see how many values in the list
len(list(names.values()))

65810

In [76]:
round(pricing_df["ECU"]/pricing_df["vCPU"],2)

7           NaN
26          NaN
40          NaN
50         3.25
60         3.25
           ... 
1603740     NaN
1603777     NaN
1603778    3.00
1603780     NaN
1603817    3.51
Length: 65810, dtype: float64

In [ ]:
#look at ratio of ECU to vCPU
pricing_df["Num_ECU_vCPU_ratio"] = round(pricing_df["ECU"]/pricing_df["vCPU"],2)

sns.set_context('paper')
plt.figure(figsize=(30, 15))
sns.set_style({'xtick.major.size': 1})

# group our dataframe so axis output is ordered nicely
# adding this option to the sns.swarmplot is very slow: order=sorted(list(names))
pricing_df = pricing_df.sort_values(['Instance Type', 'ECU'], ascending=[False, True])

fig = sns.swarmplot(x="Num_ECU_vCPU_ratio", y="Instance Type", hue="vCPU", data=pricing_df, dodge=True)
sns.despine(left=False, bottom=True)
plt.title("AWS Ratio of the # of ECU to # of vCPU for all Instance Types")
plt.xlabel("# of ECU to # of vCPU")
plt.ylabel("Instance Type")

plt.show(fig)